In [1]:
import yaml
from trainer import Trainer
import learners
import pickle

In [2]:
def get_args(path):
    class Namespace:
        def __init__(self, **kwargs):
            self.__dict__.update(kwargs)
            
        def __repr__(self):
            return str(self.__dict__)
    
    with open(path, 'r') as f:
        args = Namespace(**yaml.safe_load(f))
    return args    

In [ ]:
args = get_args('outputs/cifar-100/10-task/coda-p/args.yaml')

seed = 0
metric_keys = ['acc','time',]
save_keys = ['global', 'pt', 'pt-local']

trainer = Trainer(args, seed, metric_keys, save_keys)
self = trainer

self.learner = learners.__dict__[self.learner_type].__dict__[self.learner_name](self.learner_config)

metric_table = {}
metric_table_local = {}

for i in range(self.max_task):

    # increment task id in prompting modules
    if i > 0:
        try:
            if self.learner.model.module.prompt is not None:
                self.learner.model.module.prompt.process_task_count()
        except:
            if self.learner.model.prompt is not None:
                self.learner.model.prompt.process_task_count()

    # load model
    model_save_dir = self.model_top_dir + '/models/repeat-'+str(self.seed+1)+'/task-'+self.task_names[i]+'/'
    self.learner.task_count = i 
    self.learner.add_valid_output_dim(len(self.tasks_logits[i]))
    self.learner.pre_steps()
    self.learner.load_model(model_save_dir)

    # set task id for model (needed for prompting)
    try:
        self.learner.model.module.task_id = i
    except:
        self.learner.model.task_id = i

    self.reset_cluster_labels = True
    for j in range(i+1):
        if not i in metric_table:
            metric_table[i] = {}
        metric_table[i][j] = self.task_eval(j, return_logits=True)

    for j in range(i+1):
        if not i in metric_table_local:
            metric_table_local[i] = {}
        metric_table_local[i][j] = self.task_eval(j, local=True, return_logits=True)


Files already downloaded and verified
learner config: {'num_classes': 100, 'lr': 0.001, 'debug_mode': False, 'momentum': 0.9, 'weight_decay': 0, 'schedule': [20], 'schedule_type': 'cosine', 'model_type': 'zoo', 'model_name': 'vit_pt_imnet', 'optimizer': 'Adam', 'gpuid': [0], 'memory': 0, 'temp': 2.0, 'out_dim': 100, 'overwrite': False, 'DW': False, 'batch_size': 32, 'upper_bound_flag': False, 'tasks': [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [20, 21, 22, 23, 24, 25, 26, 27, 28, 29], [30, 31, 32, 33, 34, 35, 36, 37, 38, 39], [40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [50, 51, 52, 53, 54, 55, 56, 57, 58, 59], [60, 61, 62, 63, 64, 65, 66, 67, 68, 69], [70, 71, 72, 73, 74, 75, 76, 77, 78, 79], [80, 81, 82, 83, 84, 85, 86, 87, 88, 89], [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]], 'top_k': 1, 'prompt_param': [10, [100.0, 8.0, 0.0]]}
*****************************************
*****************************************
Incremental class: Old valid output dimension:

In [ ]:
with open('metric_table.pkl', 'wb') as f:
    pickle.dump(metric_table, f)
    
with open('metric_table_local.pkl', 'wb') as f:
    pickle.dump(metric_table_local, f)

In [2]:
# args = get_args('outputs/cifar-100/10-task/coda-p/args.yaml')

# num_classes = 100
# tasks = []
# tasks_logits = []
# class_order = np.arange(num_classes).tolist()
# class_order_logits = np.arange(num_classes).tolist()
# p = 0
# while p < num_classes and (args.max_task == -1 or len(tasks) < args.max_task):
#     inc = args.other_split_size if p > 0 else args.first_split_size
#     tasks.append(class_order[p:p+inc])
#     tasks_logits.append(class_order_logits[p:p+inc])
#     p += inc
    
# num_tasks = len(tasks)
# task_names = [str(i+1) for i in range(num_tasks)]

# # number of tasks to perform
# if args.max_task > 0:
#     max_task = min(args.max_task, len(task_names))
# else:
#     max_task = len(task_names)
    
# learner_config = {
#     'num_classes': num_classes,
#     'lr': args.lr,
#     'debug_mode': args.debug_mode == 1,
#     'momentum': args.momentum,
#     'weight_decay': args.weight_decay,
#     'schedule': args.schedule,
#     'schedule_type': args.schedule_type,
#     'model_type': args.model_type,
#     'model_name': args.model_name,
#     'optimizer': args.optimizer,
#     'gpuid': args.gpuid,
#     'memory': args.memory,
#     'temp': args.temp,
#     'out_dim': num_classes,
#     'overwrite': args.overwrite == 1,
#     'DW': args.DW,
#     'batch_size': args.batch_size,
#     'upper_bound_flag': args.upper_bound_flag,
#     'tasks': tasks_logits,
#     'top_k': 1,
#     'prompt_param':[num_tasks, args.prompt_param]
# }
# print(learner_config)
# learner_type, learner_name = args.learner_type, args.learner_name
# learner = learners.__dict__[learner_type].__dict__[learner_name](learner_config)

# Dataset = dataloaders.iCIFAR100
# dataset_size = [32,32,3]
# test_transform  = dataloaders.utils.get_transform(dataset=args.dataset, phase='test', aug=args.train_aug, resize_imnet=False)
# test_dataset  = Dataset(args.dataroot, train=False, tasks=tasks,
#                         download_flag=False, transform=test_transform, 
#                         seed=2024, rand_split=args.rand_split, validation=args.validation)


# def task_eval(t_index, local=False, task='acc'):
#     val_name = task_names[t_index]
#     print('validation split name:', val_name)
    
#     # eval
#     test_dataset.load_dataset(t_index, train=True)
#     test_loader  = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False, num_workers=4)
#     if local:
#         return learner.validation(test_loader, task_in = tasks_logits[t_index], task_metric=task)
#     else:
#         return learner.validation(test_loader, task_metric=task)

# acc_table = task_eval(0, local=True)
# learner.load_model('outputs/cifar-100/10-task/coda-p/models/repeat-1/task-1/')

# print('acc_table:', acc_table)

{'num_classes': 100, 'lr': 0.001, 'debug_mode': False, 'momentum': 0.9, 'weight_decay': 0, 'schedule': [20], 'schedule_type': 'cosine', 'model_type': 'zoo', 'model_name': 'vit_pt_imnet', 'optimizer': 'Adam', 'gpuid': [0], 'memory': 0, 'temp': 2.0, 'out_dim': 100, 'overwrite': False, 'DW': False, 'batch_size': 32, 'upper_bound_flag': False, 'tasks': [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [20, 21, 22, 23, 24, 25, 26, 27, 28, 29], [30, 31, 32, 33, 34, 35, 36, 37, 38, 39], [40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [50, 51, 52, 53, 54, 55, 56, 57, 58, 59], [60, 61, 62, 63, 64, 65, 66, 67, 68, 69], [70, 71, 72, 73, 74, 75, 76, 77, 78, 79], [80, 81, 82, 83, 84, 85, 86, 87, 88, 89], [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]], 'top_k': 1, 'prompt_param': [10, [100.0, 8.0, 0.0]]}
*****************************************
validation split name: 1
 * Val Acc 8.222, Total time 14.86
=> Load Done
acc_table: (8.222222222222221, array([0, 8, 8, 4, 6, 2, 7, 6, 0, 8, 1

In [3]:
acc_table = {}

for model_index in range(1, 11):
    learner.load_model('outputs/cifar-100/10-task/coda-p/models/repeat-1/task-{}/'.format(model_index))
    acc_table[model_index] = []
    for task_index in range(model_index):
        acc_table[model_index].append(task_eval(task_index, local=True))

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/cifar-100/10-task/coda-p/models/repeat-1/task-1class.pth'

In [ ]:
# import pickle
# with open('outputs/cifar-100/10-task/coda-p/acc_table.pkl', 'wb') as f:
#     pickle.dump(acc_table, f)